In [1]:
import pandas as pd
import requests
from collections import defaultdict
import time

input_df = pd.read_csv('addresses.csv', index_col=0)
input_df.head()

,0
0,3DKcHmHdqcUDAX2Yi2sjX9Hauam25Nk5Ja
1,3JNZyt1ocETf9oHscQfx17XNqY3odHAjaR
2,bc1q0xur68lq587lw8aqf96f4qqcmum6sqcpyyy94a
3,3JjtnDXknvY7HWidn5wt5BdZsRdXCGEt8y
4,3J9rQSzvNThPdN5FDbJAD3N8Wdce1KF6Ms


In [3]:
btc_hashes = input_df['0'].values
addr_data = defaultdict(list)
i = 0
for btc_hash in btc_hashes[:100]: # limiting for testing
    last_tx = ''
    while(True):
        URL = "https://blockstream.info/api/address/{}/txs/chain"
        if last_tx != '':
            URL+=f'/{last_tx}'

        data = requests.get(URL.format(btc_hash))


        data = data.json()
        addr_data[btc_hash].extend(data)
        if (len(data) < 25):
            break
        last_tx = data[-1]['txid']
    i+=1
    if i % 20 == 0:
        print(f"{i}/{len(btc_hashes)}")
    # time.sleep(1)
print(f"{i}/{len(btc_hashes)}")


20/899
40/899
60/899
80/899
100/899
100/899


In [53]:
history = pd.DataFrame()

for addr in addr_data.keys():
    for tx in addr_data[addr]:
        amount = 0
        for v_in in tx['vin']:
            if v_in['prevout']['scriptpubkey_address'] == addr:
                amount += v_in['prevout']['value']
        amount = -amount

        for v_out in tx['vout']:
            if v_out['scriptpubkey_address'] == addr:
                amount += v_out['value'] 
                
        amount /= 1e8
        
        tx_history = {
            'time': tx['status']['block_time'],
            'addr': addr,
            'value': amount,
            'txid':tx['txid']
        }
        tx_df = pd.DataFrame(tx_history, index=[0])
        history = history.append(tx_df, ignore_index=True)
#     break

history

,time,addr,value,txid
0,1589052010,3DKcHmHdqcUDAX2Yi2sjX9Hauam25Nk5Ja,-13.105460,c5fc5cd22c553207714ac3fa6f51645d47be707b6d5122...
1,1589050796,3DKcHmHdqcUDAX2Yi2sjX9Hauam25Nk5Ja,13.105460,4a72fc833a5ab16ad45627a3aa499f4b1ab44363a88dda...
2,1581792148,3DKcHmHdqcUDAX2Yi2sjX9Hauam25Nk5Ja,-50.000000,f9aaddbc3a7fd1079e4418dc2b0a8be76caa42c4127819...
3,1581789478,3DKcHmHdqcUDAX2Yi2sjX9Hauam25Nk5Ja,50.000000,0206a1c8723c7a0ef894354da3a4cdfa9787541c759a18...
4,1589052010,3JNZyt1ocETf9oHscQfx17XNqY3odHAjaR,-13.105078,c5fc5cd22c553207714ac3fa6f51645d47be707b6d5122...
...,...,...,...,...
224,1618857406,14q8cG1Mt5bUN1jytLvUzo9PpuTfCwBqK5,-0.022858,9f34c4b728977ff12910d88aff65735c1dabb70288d520...
225,1618511960,14q8cG1Mt5bUN1jytLvUzo9PpuTfCwBqK5,0.022858,0ab3664e58993cb6282158d50a5a463d34515e41f91f51...
226,1597281548,14q8cG1Mt5bUN1jytLvUzo9PpuTfCwBqK5,-0.200335,4acfce48e2cfeb03b5c309ba6d2669f30fdb54b147c002...
227,1597160347,14q8cG1Mt5bUN1jytLvUzo9PpuTfCwBqK5,0.099933,10428738c09276668c483fded58288dc1ee5b0da833746...
